# Starbucks Capstone Challenge

***

## Table of Contents
* [Project Set-up by Udacity](#project_setup)

***

## Project Set-up by Udacity<a class="anchor" id="project_setup"></a>

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

### Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

***

## Data Cleaning

### Exploring the Data

In the following cell, I load the data from json files.

In [1]:
import pandas as pd
import numpy as np
import math
import json
% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
portfolio.join(pd.get_dummies(portfolio['channels'].apply(pd.Series).stack()).sum(level=0))

,channels,difficulty,duration,id,offer_type,reward,email,mobile,social,web
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,1,1,1,0
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,1,1,1,1
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0,1,1,0,1
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,1,1,0,1
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5,1,0,0,1
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3,1,1,1,1
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2,1,1,1,1
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0,1,1,1,0
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,1,1,1,1
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2,1,1,0,1


After checking the head lines of these three dataframes, I notice that if the `transcript` dataframe is sorted first by `person` and then by `time`, then I can trace the events happening to all customers over time. I am most interested in what happens after an offer is received for each (`person`, `offer id`) pair. Observing the data set, I find that after `offer received`, three types of event could happen over time, being `offer viewed`, `transaction`, and `offer completed`. I observe something counter-intuitive: for some (`person`, `offer id`) pairs, `offer completed` happens before `offer viewed`. I think this is because of some errors in data collection. Because of this observation, I decide to focus only on whether an offer is completed or not, and ignore whether an offer is viewed or not. Again, this is due to the errors in the dataset, and I would love to explore the latter question if a better dataset is available.

In [3]:
transcript.sort_values(by=['person', 'time'], ascending=True, inplace=True)
transcript = transcript.reset_index(drop=True)
transcript.head(20)

,event,person,time,value
0,offer received,0009655768c64bdeb2e877511632db8f,168,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
1,offer viewed,0009655768c64bdeb2e877511632db8f,192,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
2,transaction,0009655768c64bdeb2e877511632db8f,228,{'amount': 22.16}
3,offer received,0009655768c64bdeb2e877511632db8f,336,{'offer id': '3f207df678b143eea3cee63160fa8bed'}
4,offer viewed,0009655768c64bdeb2e877511632db8f,372,{'offer id': '3f207df678b143eea3cee63160fa8bed'}
5,offer received,0009655768c64bdeb2e877511632db8f,408,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'}
6,transaction,0009655768c64bdeb2e877511632db8f,414,{'amount': 8.57}
7,offer completed,0009655768c64bdeb2e877511632db8f,414,{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d...
8,offer viewed,0009655768c64bdeb2e877511632db8f,456,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'}
9,offer received,0009655768c64bdeb2e877511632db8f,504,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}


In [4]:
sum(transcript['event'] == 'offer received')

76277

### Transforming the data set

My goal is to understand what types of offer are more likely to be completed by customers in different demographic groups. To accomplish this goal, I need to create a dataframe, collecting information about whether the received offers are completed by the customers. 

While writing codes, I noticed that the dictionary keys for the offer id in column `value` are `offer_id` if the corresponding event is `offer received`, while those are `offer id` for all other events. This is tricky.

In [ ]:
customers = [] # empty list to store the customer ids
offer_completed = [0] * transcript['event'].value_counts()['offer received'] # 1 if an offer is completed, 0 otherwise
#offer_received_times = [1] * transcript['event'].value_counts()['offer received'] # 1 if an offer is completed, 0 otherwise
#offer_viewed = [] # 1 if an offer is viewed, 0 otherwise
offer_ids = [] # to store the offer id
time_offered = [] # to store the time when offer is received
index = -1 # to trace the number of the (person, offer id) pair

for i in np.arange(transcript.shape[0]):
    if transcript.iloc[i]['event'] == 'offer completed':
        index += 1
        customer = transcript.iloc[i]['person']
        offer_id = transcript.iloc[i]['value']['offer_id']
        j = i - 1
        while transcript.iloc[j]['person'] == customer:
            if transcript.iloc[j]['event'] == 'offer received' and transcript.iloc[i]['value']['offer_id'] == transcript.iloc[j]['value']['offer id']:
                customer.append(customer)
                offer_ids.append(offer_id)
                offer_completed[index] = 1
                

In [170]:
customers = [] # empty list to store the customer ids
offer_completed = [0] * transcript['event'].value_counts()['offer received'] # 1 if an offer is completed, 0 otherwise
#offer_received_times = [1] * transcript['event'].value_counts()['offer received'] # 1 if an offer is completed, 0 otherwise
#offer_viewed = [] # 1 if an offer is viewed, 0 otherwise
offer_ids = [] # to store the offer id
#time_offered = [] # to store the time when offer is received
index = -1 # to trace the number of the (person, offer id) pair

for i in np.arange(825, 845):
    if transcript.iloc[i]['event'] == 'offer received':
        index += 1
        customer = transcript.iloc[i]['person']
        offer_id = transcript.iloc[i]['value']['offer id']
        transcript[transcript['person'] == customer and \
                   any(tmp == 'offer_id' or tmp == 'offer id' for tmp in transcript.iloc[844]['value'].keys()) ]
        
        
        j = i+1 # from the next index on, until the customer changes, check whether the particular offer is completed
        while j < transcript.shape[0] and transcript.iloc[j]['person'] == transcript.iloc[i]['person']:
            if transcript.iloc[j]['event'] == 'offer completed' and transcript.iloc[j]['value']['offer_id'] == transcript.iloc[i]['value']['offer id']:
                offer_completed[index] = 1
                print(i, index, j, transcript.iloc[j]['event'])
                break 
            j += 1

825 0 828 offer completed
831 2 836 offer completed
833 3 835 offer completed
841 4 844 offer completed


In [164]:
df = pd.DataFrame(list(zip(customer, time_offered, offer_id, offer_completed)), \
                  columns = ['customer', 'time_offered', 'offer_id', 'offer_completed'])
# check that indeed I have counted the correct numbers of `offer received` and `offer completed`
print('The number of offer received in dataset transcript is ', sum(transcript['event'] == 'offer received'),'\n'\
      'The number of offer received in my df is ', df.shape[0], '\n'\
      'The number of offer completed in dataset transcript is ', sum(transcript['event'] == 'offer completed'),'\n'\
      'The number of offer completed in my df is ', df['offer_completed'].sum(),'\n'\
      'So I have collected the information into df correctly! ')

The number of offer received in dataset transcript is  76277 
The number of offer received in my df is  76277 
The number of offer completed in dataset transcript is  33579 
The number of offer completed in my df is  33101 
So I have collected the information into df correctly! 


In [128]:
df[df['offer_completed'] == 1]['customer'].value_counts(ascending = True).reset_index()

,index,customer
0,b4a313063dbf40d0b2a4e58e366d497c,1
1,c00a8b0b114a4c898cf6b61fd4c982da,1
2,5d5944be78cf49bca0263fb25c848217,1
3,153572326a8340aca794bd6ff01b24c1,1
4,bf0e5768905743ed8752bb3b12be1463,1
5,632b544911b5482c89d73988161ea899,1
6,2149edc8436c49bf8ed9712ca0373b5a,1
7,94e1eef711c34fcd9661a5703b3ce214,1
8,82f1d31dc1a340f58262316928882210,1
9,5e78586d062041f1bfe97eacc63e1485,1


In [165]:
tmp = transcript[transcript['event'] == 'offer completed']['person'].value_counts(ascending=True).to_frame().reset_index()
tmp2 = df[df['offer_completed'] == 1]['customer'].value_counts(ascending = True).to_frame().reset_index()

In [166]:
tmp3 = tmp.merge(tmp2)
customers = tmp3[tmp3['person'] != tmp3['customer']]['index'].tolist()
tmp3[tmp3['person'] != tmp3['customer']].sort_values(by=['index'])

,index,person,customer
12557,00d7c95f793a4212af44e632fdc1e431,5,4
7074,01925607d99c460996c281f17cdbb9e2,3,2
11678,01956670cf414b309675aa73368b94a9,5,4
11273,01ff6c5d8d014dbd8c120e2b43a065ea,4,3
12728,0200f61c69da4c2ea078842cdaf234e6,6,5
4663,0246e7cc7a3d4a98a940cb13776b5b55,2,1
4447,0246f8fdf0b64014a98822b70231c58d,2,1
9773,034b962ed61d4da1bb0baaec84cc2e85,4,3
11800,03b326a99f5345df8b4d21ed26bdb0da,5,4
8441,041967ceb80841ddbd306958d4f744ac,3,2


In [39]:
transcript[transcript['person']=='00d7c95f793a4212af44e632fdc1e431']['value']

825     {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
826     {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
827                                    {'amount': 29.39}
828    {'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0...
829                                    {'amount': 20.69}
830     {'offer id': '2906b810c7d4411798c6938adc9daaa5'}
831     {'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
832     {'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
833     {'offer id': '2906b810c7d4411798c6938adc9daaa5'}
834                                    {'amount': 18.58}
835    {'offer_id': '2906b810c7d4411798c6938adc9daaa5...
836    {'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...
837    {'offer_id': '2906b810c7d4411798c6938adc9daaa5...
838                                    {'amount': 19.85}
839                                    {'amount': 19.87}
840                                     {'amount': 29.1}
841     {'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
842     {'offer id': 'fafdcd668

In [33]:
any(tmp == 'offer_id' or tmp == 'offer id' for tmp in transcript['value'].keys()) 

False

In [32]:
a

False